In [14]:
# libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime as dt
import pickle
import xgboost as xgb

In [15]:
df=pd.read_csv('../dataset/updated_train1.csv')

In [16]:
print(df.shape)
df.head()

(252108, 28)


,store_id_date,visitors,dow,year,month,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,...,hours_sunlight,solar_radiation,deepest_snowfall,total_snowfall,avg_wind_speed,avg_vapor_pressure,avg_local_pressure,avg_humidity,avg_sea_pressure,cloud_cover
0,air_ba937bf13d40fb24_2016-01-13,25,2,2016,1,7.0,23.843750,25.0,57.0,64.0,...,8.7,10.86,0.0,0.0,1.7,4.9,1010.1,60.0,1013.1,2.5
1,air_ba937bf13d40fb24_2016-01-14,32,3,2016,1,2.0,20.292308,21.0,54.0,65.0,...,9.1,12.09,0.0,0.0,2.0,4.6,1008.9,52.0,1011.9,0.5
2,air_ba937bf13d40fb24_2016-01-15,29,4,2016,1,4.0,34.738462,35.0,61.0,65.0,...,8.1,11.67,0.0,0.0,2.5,5.7,1013.1,64.0,1016.1,6.0
3,air_ba937bf13d40fb24_2016-01-16,22,5,2016,1,6.0,27.651515,27.0,53.0,66.0,...,9.1,12.41,0.0,0.0,1.9,5.1,1015.8,54.0,1018.8,2.3
4,air_ba937bf13d40fb24_2016-01-18,6,0,2016,1,2.0,13.754386,12.0,34.0,57.0,...,1.4,2.40,6.0,6.0,3.7,7.1,995.9,95.0,998.9,7.5


In [17]:
df1=df.dropna()
df1.set_index('store_id_date',drop=True,inplace=True)

In [18]:
print(df1.shape)
df1.head()

(252108, 27)


,visitors,dow,year,month,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,...,hours_sunlight,solar_radiation,deepest_snowfall,total_snowfall,avg_wind_speed,avg_vapor_pressure,avg_local_pressure,avg_humidity,avg_sea_pressure,cloud_cover
store_id_date,,,,,,,,,,,,,,,,,,,,,
air_ba937bf13d40fb24_2016-01-13,25,2,2016,1,7.0,23.843750,25.0,57.0,64.0,4.0,...,8.7,10.86,0.0,0.0,1.7,4.9,1010.1,60.0,1013.1,2.5
air_ba937bf13d40fb24_2016-01-14,32,3,2016,1,2.0,20.292308,21.0,54.0,65.0,4.0,...,9.1,12.09,0.0,0.0,2.0,4.6,1008.9,52.0,1011.9,0.5
air_ba937bf13d40fb24_2016-01-15,29,4,2016,1,4.0,34.738462,35.0,61.0,65.0,4.0,...,8.1,11.67,0.0,0.0,2.5,5.7,1013.1,64.0,1016.1,6.0
air_ba937bf13d40fb24_2016-01-16,22,5,2016,1,6.0,27.651515,27.0,53.0,66.0,4.0,...,9.1,12.41,0.0,0.0,1.9,5.1,1015.8,54.0,1018.8,2.3
air_ba937bf13d40fb24_2016-01-18,6,0,2016,1,2.0,13.754386,12.0,34.0,57.0,4.0,...,1.4,2.40,6.0,6.0,3.7,7.1,995.9,95.0,998.9,7.5


In [19]:
X = df1.drop('visitors', axis=1).as_matrix()
y = df1['visitors']

#データをわける
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 666)

In [20]:
#A function to calculate Root Mean Squared Logarithmic Error (RMSLE)
def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    terms_to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5

In [ ]:
# parameter tuning of xgboost
# start from default setting
boost_params = {'eval_metric': 'rmse'}
xgb0 = xgb.XGBRegressor(
    max_depth=8,
    learning_rate=0.01,
    n_estimators=10000,
    objective='reg:linear',
    gamma=0,
    min_child_weight=1,
    subsample=1,
    colsample_bytree=1,
    scale_pos_weight=1,
    seed=27,
    **boost_params)

xgb0.fit(X_train, y_train)
y_test_pred = xgb0.predict(X_test)
test['visitors'] = np.expm1(y_test_pred)
test[['id', 'visitors']].to_csv('xgb0_submission.csv', index=False, float_format='%.3f')  # LB0.495

In [ ]:
from sklearn.metrics import mean_squared_error
print(rmsle(y_test,y_test_pred))
print(rmsle(y_train,y_train_pred))

In [ ]:
from sklearn.metrics import r2_score
print(r2_score(y_train,y_train_pred))
print(r2_score(y_test,y_test_pred))

In [ ]:
u_lim=150
l_lim=0
plt.figure(figsize=(8, 8))
plt.plot([l_lim,u_lim], [l_lim,u_lim], c='k', alpha=0.6, label = 'diagonal line') # diagonal line
plt.scatter(y_train,y_train_pred,label='train')
plt.scatter(y_test,y_test_pred,label='test')
plt.xlim(l_lim,u_lim)
plt.ylim(l_lim,u_lim)
plt.xlabel('acutual', fontsize=16)
plt.ylabel('predicted', fontsize=16)
plt.legend()
plt.grid()
plt.show()

In [ ]:
with open('xgb.pickle', mode='wb') as f:
    pickle.dump(gs, f)

In [ ]:
import seaborn as sns
plt.figure(figsize=(10,10))
sns.heatmap(df1.corr())
plt.show()